In [0]:
#importing all the libraries
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn import svm

In [2]:
#data
import urllib.request
urllib.request.urlretrieve ("https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", "a.tar.gz")

('a.tar.gz', <http.client.HTTPMessage at 0x7fa97f1ccd68>)

In [0]:

import tarfile
tar = tarfile.open("a.tar.gz")
tar.extractall()
tar.close()

# Loading the dataset 

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pickle


def unpickle(file):
 '''Load byte data from file'''
 with open(file, 'rb') as f:
  data = pickle.load(f, encoding='latin-1')
  return data


def load_cifar10_data(data_dir):
 '''Return train_data, train_labels, test_data, test_labels
 The shape of data is 32 x 32 x3'''
 train_data = None
 train_labels = []

 for i in range(1, 6):
  data_dic = unpickle(data_dir + "/data_batch_{}".format(i))
  if i == 1:
   train_data = data_dic['data']
  else:
   train_data = np.vstack((train_data, data_dic['data']))
  train_labels += data_dic['labels']

 test_data_dic = unpickle(data_dir + "/test_batch")
 test_data = test_data_dic['data']
 test_labels = test_data_dic['labels']

 train_data = train_data.reshape((len(train_data), 3, 32, 32))
 train_data = np.rollaxis(train_data, 1, 4)
 train_labels = np.array(train_labels)

 test_data = test_data.reshape((len(test_data), 3, 32, 32))
 test_data = np.rollaxis(test_data, 1, 4)
 test_labels = np.array(test_labels)

 return train_data, train_labels, test_data, test_labels

data_dir = 'cifar-10-batches-py'

train_data, train_labels, test_data, test_labels = load_cifar10_data(data_dir)

print(train_data.shape)
print(train_labels.shape)

print(test_data.shape)
print(test_labels.shape)

(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [5]:
#getting training and testing values
xtrain = train_data.reshape(train_data.shape[0],-1)
xtest = test_data.reshape(test_data.shape[0],-1)
xtrain.shape,xtest.shape

((50000, 3072), (10000, 3072))

In [6]:
#scaling the data
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
xtrain_scaled = scaler.fit_transform(xtrain)
xtest_scaled = scaler.transform(xtest)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [7]:
ytrain = train_labels
ytest = test_labels

ytrain.shape,ytest.shape

((50000,), (10000,))

In [0]:
pca = PCA()
xtrain_pca = pca.fit_transform(xtrain_scaled)

In [12]:
#calculating the optimal value of k(num of components)
k = 0
total_calc_var = 0
while total_calc_var < .95:
  var = pca.explained_variance_ratio_[k]
  total_calc_var += var
  k += 1
k

221

In [0]:
pca = PCA(n_components=k,whiten=True)
x_train_pca = pca.fit_transform(xtrain_scaled)
x_test_pca = pca.transform(xtest_scaled)

In [0]:
svc = svm.SVC(C= 100, gamma=0.005)

In [16]:
svc.fit(x_train_pca,ytrain)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.005, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [18]:
y_pred = svc.predict(x_test_pca)
accuracy_score(ytest,y_pred)

0.571

In [30]:
y_pred_val = ["" for i in range(10000)]
for i in range(len(y_pred)):
    if(y_pred[i] == 0):
        y_pred_val[i] = "airplane"
    elif(y_pred[i] == 1):
        y_pred_val[i] = "automobile"
    elif(y_pred[i] == 2):
        y_pred_val[i] = "bird"
    elif(y_pred[i] == 3):
        y_pred_val[i] = "cat"
    elif(y_pred[i] == 4):
        y_pred_val[i] = "deer"
    elif(y_pred[i] == 5):
        y_pred_val[i] = "dog"
    elif(y_pred[i] == 6):
        y_pred_val[i] = "frog"
    elif(y_pred[i] == 7):
        y_pred_val[i] = "horse"
    elif(y_pred[i] == 8):
        y_pred_val[i] = "ship"
    else:
        y_pred_val[i] = "truck"
y_pred_val

['cat',
 'automobile',
 'airplane',
 'airplane',
 'cat',
 'frog',
 'automobile',
 'frog',
 'deer',
 'automobile',
 'airplane',
 'truck',
 'bird',
 'horse',
 'truck',
 'ship',
 'cat',
 'cat',
 'ship',
 'frog',
 'horse',
 'bird',
 'airplane',
 'truck',
 'deer',
 'frog',
 'bird',
 'deer',
 'truck',
 'frog',
 'frog',
 'deer',
 'bird',
 'cat',
 'truck',
 'automobile',
 'frog',
 'truck',
 'truck',
 'dog',
 'deer',
 'frog',
 'cat',
 'frog',
 'airplane',
 'truck',
 'dog',
 'cat',
 'deer',
 'bird',
 'truck',
 'ship',
 'dog',
 'cat',
 'ship',
 'ship',
 'horse',
 'ship',
 'cat',
 'deer',
 'horse',
 'cat',
 'frog',
 'truck',
 'frog',
 'frog',
 'automobile',
 'airplane',
 'dog',
 'truck',
 'deer',
 'dog',
 'ship',
 'ship',
 'airplane',
 'bird',
 'truck',
 'cat',
 'cat',
 'ship',
 'ship',
 'automobile',
 'automobile',
 'horse',
 'airplane',
 'horse',
 'cat',
 'airplane',
 'ship',
 'truck',
 'ship',
 'dog',
 'ship',
 'frog',
 'deer',
 'horse',
 'frog',
 'airplane',
 'airplane',
 'horse',
 'deer',
 'd

In [0]:
np.savetxt("prediction_final.csv",y_pred_val,delimiter=",",fmt="%s")

In [0]:
from google.colab import files
files.download("prediction_final.csv")

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}